# Guided Investigation - Anomaly Lookup

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0, azure-cli-profile 2.1.4<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebook on DSVM
     
__Data Source Required:__<br>
    -  Log Analytics tables 
    
### Description
Gain insights into the possible root cause of an alert by searching for related anomalies on the corresponding entities around the alert’s time. This notebook will provide valuable leads for an alert’s investigation, listing all suspicious increase in event counts or their properties around the time of the alert, and linking to the corresponding raw records in Log Analytics for the investigator to focus on and interpret.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Initialize Azure Resource Management Clients
2. Looking up for anomaly entities

## 1. Initialize Azure Resource Management Clients

In [ ]:
# only run once
!pip install --upgrade Azure-Sentinel-Utilities

In [ ]:
# User Input and Save to Environmental store
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
import SentinelAzure
from SentinelWidgets import *
from SentinelAnomalyLookup import *

from pandas.io.json import json_normalize
from azure.loganalytics import LogAnalyticsDataClient
from azure.loganalytics.models import QueryBody
import sys
import timeit
import datetime as dt
import pandas as pd
import copy
from IPython.display import HTML

# User Input and Save to Environmental store
# Set to True if you want to reset the environmental values
import os
from SentinelWidgets import *
reset_env_vars = False
env_dir = %env
env_list = ['tenant_domain', 'subscription_id', 'resource_group']
envs =  {env_list[0]:'', env_list[1]:'', env_list[2]:''}
envs = WidgetViewHelper.set_env(reset_env_vars, env_dir, envs)
tenant_domain = envs[env_list[0]]
subscription_id = envs[env_list[1]]
resource_group = envs[env_list[2]]

In [ ]:
# Authentication to Log Analytics
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant $tenant_domain
la_client = get_client_from_cli_profile(LogAnalyticsManagementClient, subscription_id = subscription_id)
la = SentinelAzure.azure_loganalytics_helper.LogAnalyticsHelper(la_client)
creds, _ = get_azure_cli_credentials(resource="https://api.loganalytics.io")
la_data_client = LogAnalyticsDataClient(creds)

## 2. Looking up for anomaly entities

In [ ]:
# Select a workspace
selected_workspace = WidgetViewHelper.select_log_analytics_workspace(la)
display(selected_workspace)

In [ ]:
workspace_id = la.get_workspace_id(selected_workspace.value)
#DateTime format: 2019-07-15T07:05:20.000
q_timestamp = input('DateTime: ')
#Entity format: computer
q_entity = input('Entity for search: ')

In [ ]:
anomaly_lookup = AnomalyFinder(workspace_id, la_data_client)
selected_tables = WidgetViewHelper.select_multiple_tables(anomaly_lookup)
display(selected_tables)

In [ ]:
# This action may take a few minutes or more, please be patient.
start = timeit.default_timer()
anomalies, queries = anomaly_lookup.run(q_timestamp, q_entity, list(selected_tables.value))
display(anomalies)

if queries is not None:
    url = WidgetViewHelper.construct_url_for_log_analytics_logs(tenant_domain, subscription_id, resource_group, selected_workspace.value)
    WidgetViewHelper.display_html(WidgetViewHelper.copy_to_clipboard(url, queries, 'Add queries to clipboard and go to Log Analytics'))

print('==================')
print('Elapsed time: ', timeit.default_timer() - start, ' seconds')